<a href="https://colab.research.google.com/github/reuven-itzhakov/Cloud-Computing/blob/main/HW2_Sloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Do not Run Again
# def get_sublinks(soup):
#     root = 'https://mqtt.org/'
#     sublinks = set()
#     for link in soup.find_all('a', **{'href': True}):
#         if link['href'].startswith('/') and link['href'][-4:] not in ['.zip','.png']:
#             if link['href'].find('#') != -1:
#                 link['href'] = link['href'][:link['href'].find('#')]
#             link['href'] = root + link['href'][1:]
#             sublinks.add(link['href'])
#     return sublinks

# index = dict()
# stemmer = Stemmer()
# stopwords = {
#     "a", "an", "the", "and", "or", "but", "if", "while", "with",
#     "to", "from", "in", "on", "at", "by", "for", "of", "up", "down",
#     "out", "over", "under", "is", "are", "was"
# }

# root = ('https://mqtt.org/', 0)
# links_to_visit = Queue()
# links_to_visit.put(root)
# links_visited = set()

# def scrape(soup,link):
#     words = re.findall(r'\w+', soup.get_text())
#     wc = dict()
#     for word in words:
#         word = word.lower()
#         if word not in stopwords:
#             stemmed_word = stemmer.stem(word)
#             wc[stemmed_word] = wc.get(stemmed_word, 0) + 1

#     for word, count in wc.items():
#         if count > 5:
#             dic = index.get(word, dict())
#             dic[link] = count
#             index[word] = dic

# while not links_to_visit.empty():
#     link, depth = links_to_visit.get()
#     # Mark as visited immediately
#     links_visited.add(link[:-1] if link.endswith('/') else link)
#     print(depth, link)
#     time.sleep(0.01)
#     response = requests.get(link)
#     soup = bs(response.text, 'html.parser')
#     scrape(soup,link)
#     if depth < 2:
#         # Simplified enqueueing
#         for l in get_sublinks(soup):
#             if (l[:-1] if l.endswith('/') else l) not in links_visited:
#                 links_to_visit.put((l, depth + 1))
#                 links_visited.add(l[:-1] if l.endswith('/') else l)  # Mark as visited after enqueueing

# for key, value in index.items():
#     item ={
#             "term":key,
#             "DocIDs":{ i:l for i,l in enumerate(value.keys(),start=1)},
#             "DocCount":{i:l for i,l in enumerate(value.values(),start=1)}
#     }
#     FBconn.put("/hwIndex/",key,item)

In [2]:
!pip install requests beautifulsoup4 nltk firebase paho-mqtt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.7 MB/s eta 0:00:00


In [3]:
import IPython
from ipywidgets import Dropdown, Output, Text, Button, VBox, Label, widgets
from IPython.display import display, clear_output, HTML
from firebase import firebase

import requests
import pandas as pd
from plotly.offline import iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import numpy as np
# import seaborn as sns # (irrelevant)
# import plotly.express as px # (irrelevant)
import threading
import time
import random
from datetime import datetime
import pytz
from zoneinfo import ZoneInfo

from bs4 import BeautifulSoup as bs
import nltk
from nltk.stem import PorterStemmer as Stemmer
from queue import Queue
import re
import paho.mqtt.client as mqtt
import json

In [4]:
FBconn = firebase.FirebaseApplication(
    'https://cloud-comp-sloth-default-rtdb.europe-west1.firebasedatabase.app/',
    None
)

In [5]:
# --- Constants and Setup : ---
STOP_WORDS = {
    "a", "an", "the", "and", "or", "but", "if", "while", "with",
    "to", "from", "in", "on", "at", "by", "for", "of", "up", "down",
    "out", "over", "under", "is", "are", "was"
}
STEMMER = Stemmer()

# Global variables for the index and document mappings
inverted_index = {}
documents_map = {} # Will store { "fb_doc_1": "url1", ... }
document_titles = {} # Will store { "fb_doc_1": "title_or_url1", ... }

In [6]:
def preprocess_text(text):
    if not text:
        return []
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text) # Finds whole words

    processed_tokens = []
    for token in tokens:
        if token not in STOP_WORDS:
            processed_tokens.append(STEMMER.stem(token))
    return processed_tokens

In [7]:
# --- Function to Load and Transform Index from Firebase (REVISED FOR KEY NORMALIZATION) ---
def load_index_from_firebase():
    global inverted_index, documents_map, document_titles

    inverted_index = {}
    documents_map = {}
    document_titles = {}


    with results_output_area:
        clear_output(wait=True)
        print("Attempting to load hwIndex from Firebase...")

    firebase_hw_index_data = FBconn.get("/hwIndex", None)

    if not firebase_hw_index_data or not isinstance(firebase_hw_index_data, dict):
        print("Error: Could not fetch hwIndex from Firebase, it's empty, or not in expected dictionary format.")
        return

    for term_from_fb, term_data_obj in firebase_hw_index_data.items():
        if not isinstance(term_data_obj, dict):
            print(f"Warning: Skipping term entry for '{term_from_fb}' as it's not a dictionary.")
            continue


        # This ensures it matches the lowercase processing of query terms
        current_index_term = term_from_fb.lower()

        if not current_index_term: # If term becomes empty after lowercasing (unlikely for valid terms)
            print(f"Warning: Term '{term_from_fb}' became empty after lowercasing. Skipping.")
            continue

        firebase_doc_ids = term_data_obj.get('DocIDs')
        firebase_doc_counts = term_data_obj.get('DocCount')

        if current_index_term not in inverted_index:
            inverted_index[current_index_term] = {}

        if isinstance(firebase_doc_ids, dict) and isinstance(firebase_doc_counts, dict):
            for internal_fb_doc_key, doc_url in firebase_doc_ids.items():
                python_doc_id = f"fb_doc_dict_{internal_fb_doc_key}"
                if python_doc_id not in documents_map:
                    documents_map[python_doc_id] = doc_url
                    document_titles[python_doc_id] = doc_url
                term_frequency_in_doc = firebase_doc_counts.get(internal_fb_doc_key)
                if term_frequency_in_doc is not None:
                    inverted_index[current_index_term][python_doc_id] = term_frequency_in_doc
                else:
                    print(f"Warning: Missing DocCount for dict key '{internal_fb_doc_key}' under term '{current_index_term}'.")

        elif isinstance(firebase_doc_ids, list) and isinstance(firebase_doc_counts, list):
            if len(firebase_doc_ids) != len(firebase_doc_counts):
                print(f"Warning: Mismatch in lengths for DocIDs list ({len(firebase_doc_ids)}) and DocCount list ({len(firebase_doc_counts)}) for term '{current_index_term}'. Skipping this term's list data.")
                continue
            for i, doc_url in enumerate(firebase_doc_ids):
                if doc_url is None:
                    if i < len(firebase_doc_counts) and firebase_doc_counts[i] is not None:
                         print(f"Warning: Null URL at list index {i} for term '{current_index_term}' but count exists. Skipping this entry.")
                    continue
                python_doc_id = f"fb_doc_list_{i}"
                if python_doc_id not in documents_map:
                    documents_map[python_doc_id] = doc_url
                    document_titles[python_doc_id] = doc_url
                term_frequency_in_doc = firebase_doc_counts[i]
                if term_frequency_in_doc is not None:
                    inverted_index[current_index_term][python_doc_id] = term_frequency_in_doc
                else:
                    print(f"Warning: Null DocCount at list index {i} for term '{current_index_term}' with URL '{doc_url}'.")
        else:
            print(f"Warning: Skipping term '{current_index_term}' (original: '{term_from_fb}') due to unexpected or mismatched format for DocIDs/DocCount.")
            continue



    num_terms = len(inverted_index)
    num_docs = len(documents_map)
    with results_output_area:
        clear_output(wait=True)
        print("Index loaded. Ready for search.")


In [8]:
# --- Search Function : ---
def search_in_index(query):
    # query_tokens will be processed based on stem_query_flag
    query_tokens = preprocess_text(query)

    if not query_tokens:
        return []

    doc_scores = {} # Using a dictionary

    for token in query_tokens:
        # Search for the (stemmed) token in our loaded (stemmed) index
        if token in inverted_index:
            for doc_id, freq in inverted_index[token].items():
                if doc_id not in doc_scores:
                    doc_scores[doc_id] = {'match_count': 0, 'total_frequency': 0}
                doc_scores[doc_id]['match_count'] += 1
                doc_scores[doc_id]['total_frequency'] += freq

    # Sort results: primary key match_count, secondary key total_frequency
    sorted_docs_items = sorted(doc_scores.items(),
                               key=lambda item: (item[1]['match_count'], item[1]['total_frequency']),
                               reverse=True)

    results = []
    for doc_id, scores in sorted_docs_items:
        results.append({
            "doc_id": doc_id,
            "url": documents_map.get(doc_id, "URL not found"),
            "title": document_titles.get(doc_id, "Title not found"), # Will be URL if not otherwise available
            "score_details": scores
        })
    return results


In [9]:
# --- UI Elements and Handlers : ---
query_input = widgets.Text(value='', placeholder='Enter search query...', description='Query:', layout=widgets.Layout(width='50%'))
search_button = widgets.Button(description='Search', button_style='info', tooltip='Click to search', icon='search')
# "Rebuild Index" button will now "Reload Index from Firebase"
reload_index_button = widgets.Button(description='Reload Index from Firebase', button_style='warning', tooltip='Click to reload index from Firebase', icon='refresh')

results_output_area = widgets.Output()


def handle_search_button_click(b):
    query_text = query_input.value
    if not query_text:
        with results_output_area: clear_output(wait=True); print("Please enter a query.")
        return
    if not inverted_index:
        with results_output_area: clear_output(wait=True); print("Index is empty. Please load it from Firebase first.")
        return

    search_results = search_in_index(query_text)

    def normalize_url(url):
       if url.endswith('/'):
           url = url[:-1]
       return url.lower()


    seen = set()
    deduped_results = []
    for res in search_results:
         url = normalize_url(res['url'])
         if url not in seen:
             seen.add(url)
             deduped_results.append(res)


    with results_output_area:
      clear_output(wait=True)
      if deduped_results:
          print(f"Found {len(deduped_results)} results for '{query_text}':\n")
          for i, res in enumerate(deduped_results):
              html_link = f"<p><b>{i+1}. <a href='{res['url']}' target='_blank'>{res['title']}</a></b><br>"
              html_link += f"<small>URL: {res['url']}<br>"
              html_link += f"Matching query terms: {res['score_details']['match_count']}<br>"
              html_link += f"Total occurrences in document: {res['score_details']['total_frequency']}</small></p>"
              display(HTML(html_link))
      else:
          print(f"No results found for '{query_text}'.")


def handle_reload_index_button_click(b):
    # This button now triggers a reload from Firebase
    load_index_from_firebase()
    # Status label is updated within load_index_from_firebase

In [10]:
def get_search_window():
    search_button.on_click(handle_search_button_click)
    reload_index_button.on_click(handle_reload_index_button_click)

    controls_box = widgets.VBox([
        widgets.HBox([query_input, search_button]),
        widgets.HBox([reload_index_button]),
    ])

    # --- Initial Load and Display ---
    display(controls_box, results_output_area)
    # Perform an initial load when the UI is displayed
    load_index_from_firebase()

In [11]:
def get_settings():
    ret = {"dt": {},
           "vv": {}}
    ret["dt"].update(FBconn.get('/','deltaTimes'))
    ret["vv"].update(FBconn.get('/','validValues'))
    return ret

In [12]:
def fake_data(last_data={},timeout = 1):
    start_time = time.time()
    sensors = ['Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
               'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)']
    data = {}
    for sensor in sensors:
        if sensor in last_data:
            if last_data[sensor]<20:
                mu = 0.5
            elif last_data[sensor]>60:
                mu = -0.5
            else:
                mu = 0
            delta = random.gauss(mu,0.5)
            data[sensor] = last_data[sensor] + delta
        else:
            data[sensor] = random.gauss(40,2)
    time.sleep(min(abs(random.gauss(0,0.1)),timeout))
    data["timestamp"] = int(time.time())
    return data

In [13]:
sensors = ['Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
           'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)']
units = {
    "Humidity(Outdoor)":"%",
    "Pressure(Outdoor)":"Pa",
    "Temperature(Outdoor)":"°C",
    "Dlight(Outdoor)":"lx",
    "Humidity(Indoor)":"%",
    "Pressure(Indoor)":"Pa",
    "Temperature(Indoor)":"°C",
    "Distance(Indoor)":"mm"
}
sensor_units = units
fake = fake_data(timeout=1)
temps = {}
for sensor in sensors:
    temps[sensor] = [fake[sensor]]
tab = widgets.Output()
chosen_sensor = sensors[0]

def update(settings,t=1):
    min = settings["vv"][chosen_sensor]["min value"]
    max = settings["vv"][chosen_sensor]["max value"]
    delta = settings["dt"][chosen_sensor]
    data = fake_data(last_data={chosen_sensor:temps[chosen_sensor][-1]},timeout=1)[chosen_sensor]
    temps[chosen_sensor].append(data)
    if len(temps[chosen_sensor])>delta:
        temps[chosen_sensor].pop(0)
    xdata = np.arange(0, len(temps[chosen_sensor]))
    ydata = np.array(temps[chosen_sensor])
    extreme= np.concatenate((np.where(ydata>max),np.where(ydata<min)),axis=1)
    plt.plot(xdata, ydata)
    plt.plot(xdata[extreme],ydata[extreme],"ro")
    plt.plot(xdata,np.zeros_like(xdata)+max,"r--")
    plt.plot(xdata,np.zeros_like(xdata)+min,"b--")
    plt.plot(xdata,np.zeros_like(xdata)+np.average(ydata),"--",color="orange")
    plt.xlim(1,delta)
    plt.ylabel(f"{units[chosen_sensor]}")
    plt.title(chosen_sensor)
    txt = f"{data:.1f}{units[chosen_sensor]}"
    plt.text(xdata[-1]-len(txt)*1.15*xdata[-1]/delta,ydata[-1]+xdata[-1]/delta,
             txt,
             fontweight=np.max((600+150*xdata[-1]/delta, 1000)),
             fontsize=10+5*xdata[-1]/delta)

In [14]:
def change_sensor(sensor):
    global chosen_sensor
    chosen_sensor = sensor

In [15]:
def get_raw_data():
    settings = get_settings()
    p=widgets.Play(
    value=50,
    min=0,
    max=1001000,
    step=1,
    interval=1000
    )
    dd = widgets.Dropdown(
        options=sensors,
        value=sensors[0],
        description='Sensor:',
    )
    dd.observe(lambda change: change_sensor(change['new']) , 'value')
    display(dd)
    print("  ↓ Click Here Please")
    inter = widgets.interact(lambda t:update(settings,t),t=p)

In [16]:
def check_outliers_batch(selected_sensor):
    data_dict = FBconn.get(f'/sensors/{selected_sensor}', None)
    positive_infinity = float('inf')
    negative_infinity = float('-inf')

    min_value = FBconn.get(f'/validValues/{selected_sensor}/min value', None)
    max_value = FBconn.get(f'/validValues/{selected_sensor}/max value', None)

    if min_value is None:
        min_value = negative_infinity
    if max_value is None:
        max_value = positive_infinity

    min_value = float(min_value)
    max_value = float(max_value)

    outliers = {}

    for timestamp, value in data_dict.items():
        if float(value) < min_value or float(value) > max_value:
            outliers[timestamp] = value

    if outliers:
        FBconn.put(f'/outliers/', selected_sensor, outliers)

In [17]:
def view_outliers():
  print("Choose the relevant sensor to view outliers:")
  w = widgets.Dropdown(
    options=['Select', 'Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
             'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)'],
    value='Select',
    description='Sensor:',
  )
  out = Output()
  selected_sensor = w.value

  def change(change):
    global selected_sensor
    if change['type'] == 'change' and change['name'] == 'value':
      selected_sensor = change['new']
      out.clear_output()
      if selected_sensor == 'Select':
        with out:
          print("Please select a sensor.")
          return
      with out:
        print_outliers(selected_sensor)

  def print_outliers(selected_sensor):
    outliers = FBconn.get(f'/outliers/{selected_sensor}', None)
    if not outliers:
      print("No outliers found")
      return
    print("The outliers of %s sensor are: " %selected_sensor)
    for timestamp in outliers:
      readable_time = datetime.fromtimestamp(int(timestamp), ZoneInfo("Asia/Jerusalem")).strftime('%Y-%m-%d %H:%M:%S')
      print(f"  -  Timestamp: {readable_time}, Value: {outliers[timestamp]:.3f}")


  w.observe(change)
  display(w, out)

In [18]:
def get_last_max_value(sensor):
    return FBconn.get(f'/validValues/{sensor}/max value', None)

def get_last_min_value(sensor):
    return FBconn.get(f'/validValues/{sensor}/min value', None)

In [19]:
def change_valid_values():
    print("Choose the relevant sensor to update the valid values:")

    w = widgets.Dropdown(
        options=['Select', 'Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
                 'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)'],
        value='Select',
        description='Sensor:'
    )
    out = Output()

    txt_max = widgets.Text(description='Max:')
    txt_min = widgets.Text(description='Min:')
    submit_button = widgets.Button(description="Save", button_style='success')

    inputs_box = VBox([txt_max, txt_min, submit_button])
    selected_sensor = {'value': None}

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            sensor = change['new']
            selected_sensor['value'] = sensor
            out.clear_output()
            # Reset the text fields when the sensor changes
            txt_max.value = str(get_last_max_value(sensor))
            txt_min.value = str(get_last_min_value(sensor))
            if sensor == 'Select':
                with out:
                    print("Please select a sensor.")
                return

            with out:
                print(f"{sensor} selected. Please enter the valid range:")
                display(inputs_box)

    def on_submit(b):
        sensor = selected_sensor['value']
        if not sensor or sensor == 'Select':
            with out:
                print("Sensor not selected.")
            return

        max_val = txt_max.value
        min_val = txt_min.value

        if not max_val or not min_val:
            with out:
                print("Both min and max values must be filled.")
            return

        try:
            max_val = float(max_val)
            min_val = float(min_val)
        except ValueError:
            with out:
                print("Invalid input. Please enter numeric values.")
            return

        # Save to Firebase
        FBconn.put(f'/validValues/{sensor}', 'max value', max_val)
        FBconn.put(f'/validValues/{sensor}', 'min value', min_val)

        with out:
            print(f"✅ Saved min={min_val}, max={max_val} for {sensor}")

        # Now run outlier check
        check_outliers_batch(sensor)

    w.observe(on_change, names='value')
    submit_button.on_click(on_submit)

    display(w, out)

In [20]:
def try_upload_to_firebase():
  global sensor_max_value, sensor_min_value, selected_sensor

  try:
    data_to_upload = {
      'min value': (sensor_min_value),
      'max value': (sensor_max_value)
    }
    FBconn.put('/validValues', selected_sensor, data_to_upload)
  except Exception as e:
    print(f"Error uploading to Firebase: {e}")

  finally:
    sensor_max_value = None
    sensor_min_value = None

In [21]:
def get_last_delta_value(sensor):
    return FBconn.get("/deltaTimes", sensor)

In [22]:
def save_delta_time():
    print("Choose the relevant sensor to update the delta time:")

    sensor_dropdown = Dropdown(
        options=[
            'Select',
            'Humidity(Outdoor)',
            'Pressure(Outdoor)',
            'Temperature(Outdoor)',
            'Dlight(Outdoor)',
            'Humidity(Indoor)',
            'Pressure(Indoor)',
            'Temperature(Indoor)',
            'Distance(Indoor)'
        ],
        value='Select',
        description='Sensor:'
    )

    delta_input = Text(description='Delta time:')
    save_button = Button(description='Save', button_style='success')
    out = Output()

    delta_input.layout.display = 'none'
    save_button.layout.display = 'none'

    def on_sensor_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected = change['new']
            sensor = change['new']
            delta_input.value = str(get_last_delta_value(sensor))
            out.clear_output()
            if selected == 'Select':
                delta_input.layout.display = 'none'
                save_button.layout.display = 'none'
                with out:
                    print("Please select a valid sensor.")
            else:
                delta_input.layout.display = 'block'
                save_button.layout.display = 'block'
                with out:
                    print(f"{selected} selected. Please enter delta time in seconds.")

    def on_save_clicked(b):
        selected_sensor = sensor_dropdown.value
        try:
            delta_time = int(delta_input.value)
            if delta_time <= 0:
                raise ValueError("Delta time must be a positive integer.")
            FBconn.put('/deltaTimes', selected_sensor, delta_time)
            with out:
                print(f"✅ Saved delta time for {selected_sensor}: {delta_time} seconds")
        except ValueError:
            with out:
                print("❌ Invalid input. Please enter an integer.")
        except Exception as e:
            with out:
                print(f"❌ Error uploading to Firebase: {e}")

    sensor_dropdown.observe(on_sensor_change)
    save_button.on_click(on_save_clicked)

    display(VBox([
        sensor_dropdown,
        delta_input,
        save_button,
        out
    ]))
def try_upload_to_firebase_delta_time():
  global delta_time, selected_sensor
  try:
    FBconn.put('/deltaTimes', selected_sensor, delta_time)
  except Exception as e:
    print(f"Error uploading to Firebase: {e}")

In [23]:
def show_average():
  print("Choose the relevant sensor to view the average:")
  w = widgets.Dropdown(
    options=['Select','Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
             'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)'],
    value='Select',
    description='Sensor:',
  )
  sensor_units = {
        "Humidity(Outdoor)":"%",
        "Pressure(Outdoor)":"Pa",
        "Temperature(Outdoor)":"°C",
        "Dlight(Outdoor)":"lx",
        "Humidity(Indoor)":"%",
        "Pressure(Indoor)":"Pa",
        "Temperature(Indoor)":"°C",
        "Distance(Indoor)":"mm"
    }
  out = Output()
  selected_sensor = w.value
  def change(change):
    global selected_sensor
    if change['type'] == 'change' and change['name'] == 'value':
      selected_sensor = change['new']
      out.clear_output()
      if selected_sensor == 'Select':
        with out:
          print("Please select a sensor.")
          return
      with out:
        unit = sensor_units[selected_sensor]
        print(f"The average of {selected_sensor} sensor is: {average_sensor_data(selected_sensor):.3f} {unit}")
  w.observe(change)
  display(w, out)


def average_sensor_data(sensor):
  data = FBconn.get(f'/sensors/{sensor}', None)
  average = sum(data.values()) / len(data)
  return average

In [24]:
def create_graph_viewer():
    print("Choose the relevant sensor to view the statistic graph: ")
    sensor_options = [
        'Humidity(Outdoor)', 'Pressure(Outdoor)', 'Temperature(Outdoor)',
        'Dlight(Outdoor)','Humidity(Indoor)', 'Pressure(Indoor)', 'Temperature(Indoor)', 'Distance(Indoor)'
    ]

    dropdown = widgets.Dropdown(
        options=['Select a sensor'] + sensor_options,
        value='Select a sensor',
        description='Sensor:'
    )

    show_all_button = widgets.Button(
        description='Show All Sensors',
        button_style='info'
    )

    hide_all_button = widgets.Button(
        description='Hide All Graphs',
        button_style='danger'
    )

    out = Output()

    def on_dropdown_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected = change['new']
            out.clear_output()
            if selected != 'Select a sensor':
                with out:
                    generate_graph(selected, figsize=(8, 4))

    def on_show_all_clicked(b):
      show_all_button.disabled = True
      hide_all_button.disabled = True
      dropdown.disabled = True
      out.clear_output()
      with out:
          display(HTML("<p style='font-size:24px; color:red; font-weight:bold;'>Loading... Please wait</p>"))
          num_cols = 2
          num_rows = (len(sensor_options) + num_cols - 1) // num_cols
          fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 3 * num_rows))
          axes = axes.flatten()

          for i, sensor in enumerate(sensor_options):
              generate_graph(sensor, ax=axes[i])

          for j in range(i + 1, len(axes)):
            axes[j].axis('off')

          plt.subplots_adjust(hspace=2, wspace=0.5)
          out.clear_output()
          plt.show()
      show_all_button.disabled = False
      hide_all_button.disabled = False
      dropdown.disabled = False


    def on_hide_all_clicked(b):
        out.clear_output()

    dropdown.observe(on_dropdown_change)
    show_all_button.on_click(on_show_all_clicked)
    hide_all_button.on_click(on_hide_all_clicked)

    controls = widgets.HBox([dropdown, show_all_button, hide_all_button])
    display(widgets.VBox([controls, out]))


def generate_graph(sensor, ax=None, figsize=(10, 5)):
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
        show_plot = True
    else:
        show_plot = False

    data = FBconn.get(f'/sensors/{sensor}', None)
    if not data:
        ax.text(0.5, 0.5, f"No data for {sensor}", ha='center', va='center')
        ax.axis('off')
        return

    valid_min = FBconn.get(f'/validValues/{sensor}/min value', None)
    valid_max = FBconn.get(f'/validValues/{sensor}/max value', None)

    sensor_units = {
        "Humidity(Outdoor)":"%",
        "Pressure(Outdoor)":"Pa",
        "Temperature(Outdoor)":"°C",
        "Dlight(Outdoor)":"lx",
        "Humidity(Indoor)":"%",
        "Pressure(Indoor)":"Pa",
        "Temperature(Indoor)":"°C",
        "Distance(Indoor)":"mm"
    }

    unit = sensor_units.get(sensor, '')

    timestamps = list(data.keys())
    values = list(data.values())

    timestamps_filtered = timestamps[::10]
    values_filtered = values[::10]

    datetimes = [datetime.fromtimestamp(int(ts), ZoneInfo("Asia/Jerusalem")) for ts in timestamps_filtered]

    x = [dt.strftime('%H:%M:%S') for dt in datetimes]
    y = values_filtered

    date = datetimes[0].strftime('%Y-%m-%d') if datetimes else 'No Date'

    ax.plot(x, y)
    ax.set_title(f'{sensor} ({date})')
    ax.set_xlabel('Time')
    ax.set_ylabel(unit)
    ax.set_xticks(range(0, len(x), 30))
    ax.set_xticklabels(x[::30], rotation=90, fontsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.grid(True)

    if valid_min is not None:
      ax.axhline(y=valid_min, color='red', linestyle='--')
      ax.text(len(x)-1, valid_min, f'{valid_min:.2f}', color='red', va='bottom', ha='right', fontsize=9)

    if valid_max is not None:
      ax.axhline(y=valid_max, color='red', linestyle='--')
      ax.text(len(x)-1, valid_max, f'{valid_max:.2f}', color='red', va='bottom', ha='right', fontsize=9)

    if show_plot:
        plt.tight_layout()
        plt.show()

In [25]:
def on_message(client, userdata, msg):
    try:
        data = json.loads(msg.payload.decode())
        client.data = data
    except json.JSONDecodeError:
        print("Received invalid JSON data")
    client.loop_stop()

In [42]:
try:
    indoor = mqtt.Client()
    indoor.on_message = on_message
    indoor.connect("test.mosquitto.org", 1883, 60)
    stat = indoor.subscribe("braude/D106/indoor")[0]
    print('connected to indoor: ',mqtt.error_string(stat))
    outdoor = mqtt.Client()
    outdoor.on_message = on_message
    outdoor.connect("test.mosquitto.org", 1883, 60)
    stat = outdoor.subscribe("braude/D106/outdoor")[0]
    print('connected to outdoor: ',mqtt.error_string(stat))
except OSError:
    print("Connection failed. Sonsors not working.")

<ipython-input-42-17b92162f5aa>:2: DeprecationWarning:

Callback API version 1 is deprecated, update to latest version



Connection failed. Sonsors not working.


In [43]:
def real_data(last_data={},timeout = 3):
    for client in [indoor, outdoor]:
        client.loop_start()
    start_time = time.time()
    data = {'Humidity(Outdoor)': np.nan,
            'Pressure(Outdoor)': np.nan,
            'Temperature(Outdoor)': np.nan,
            'Dlight(Outdoor)': np.nan,
            'Humidity(Indoor)': np.nan,
            'Pressure(Indoor)': np.nan,
            'Temperature(Indoor)': np.nan,
            'Distance(Indoor)': np.nan}
    data.update(last_data)
    response = 0
    i = 0
    while True:
        if time.time() - start_time > timeout:
            break
        for client in [indoor, outdoor]:
            if hasattr(client, 'data'):
                data.update(client.data)
                del client.data
                response += 1
        if response == 2:
            break
        i+=1
        time.sleep(0.1)
    data["timestamp"] = int(time.time())
    return data

In [44]:
def show_availability():
    out = Output()
    refresh_button = widgets.Button(description="refresh", button_style='success')

    def check_sensors_status(_=None):
          with out:
            out.clear_output()
            print("The current status of the sensors:")
            data = real_data(timeout = 1)
            for key, value in data.items():
                if key == 'timestamp':
                    continue
                if value is np.nan:
                  print(f"  -  {key}: \033[91mnot available\033[0m")
                else:
                    print(f"  -  {key}: \033[92mavailable\033[0m")

    refresh_button.on_click(check_sensors_status)
    check_sensors_status()
    vbox = widgets.VBox([out, refresh_button])
    display(vbox)

In [45]:
# Tab 1: Administrator screen
tab1_content = widgets.Output()
with tab1_content:
  print("Administrator screen:")
  print()
  show_availability()
  print()
  print("Outlier Thresholds Explanation:")
  print(" - Use the Minimum and Maximum value fields to set the acceptable range for sensor data.")
  print(" - Any value falling outside this range will be identified as an outlier.")
  print(" - Outliers are visually highlighted in both the static and live graphs (e.g., using red markers).")
  print(" - Adjusting these values allows you to filter and monitor abnormal sensor behavior more effectively.")
  print()
  change_valid_values()
  print()
  print()
  print(" - The Delta Time setting controls how many seconds of data are shown in the live graph. For example, setting it to 30 will display the last 30 seconds.")
  print()
  save_delta_time()
  print()
  print()
  view_outliers()

# Tab 2: Raw Data
tab2_content = widgets.Output()
with tab2_content:
  print("Live Sensor Data:")
  print("This screen displays real-time data from the sensors.")
  print("The graph updates continuously and shows the most recent values.")
  print("Outliers are marked as red dots on the graph based on the defined minimum and maximum thresholds.")
  print("The time window displayed in the graph is controlled from the Administrator tab by updating the Delta Time.")
  print("For example, setting Delta Time to 30 will display only the last 30 seconds of data.\n")
  get_raw_data()

# Tab 3: Statistics
tab3_content = widgets.Output()
with tab3_content:
  print()
  print("Statistics Screen:")
  print(" - In this screen, you can analyze statistical information for each sensor.")
  print(" - You can select a sensor to view its average value based on the collected data.")
  print(" - You can also generate a statistical graph that displays the sensor's value trends over time.")
  print(" - Outliers, as defined by the minimum and maximum thresholds set in the Administrator screen, are highlighted in the graphs (e.g., with red markers).")
  print(" - These tools help identify patterns, trends, and unusual behavior in sensor readings.")
  print()
  show_average()
  print()
  create_graph_viewer()

# Tab 4: Search
tab4_content = widgets.Output()
with tab4_content:
  get_search_window()


# Create Tabs
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content, tab4_content])
tabs.set_title(0, 'Tab 1: Administrator')
tabs.set_title(1, 'Tab 2: Raw Data')
tabs.set_title(2, 'Tab 3: Statistics')
tabs.set_title(3, 'Tab 4: Search')


# Display Tabs
display(tabs)